In [1]:
"""
PREPARE BASE MODEL RESEARCH NOTEBOOK
====================================
Modern TensorFlow/Keras model preparation with best practices:
- Latest Keras 3.0+ API (.keras format, not .h5)
- Mixed precision training for better performance
- Modern optimizer configurations
- Proper model serialization
- Environment variable management
"""

import os
import sys
from pathlib import Path
import logging
from typing import Optional, Tuple

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

In [2]:
# Navigate to project root using pathlib (cross-platform)
project_root = Path(__file__).resolve().parent.parent if '__file__' in globals() else Path.cwd().parent
os.chdir(project_root)
print(f"✓ Working directory: {os.getcwd()}")

✓ Working directory: c:\Users\asus\Desktop\Deep Learning project\Chest-Cancer-Classification


In [3]:
# Load environment variables
from dotenv import load_dotenv

env_path = Path('.env')
if env_path.exists():
    load_dotenv(env_path)
    print("✓ Environment variables loaded from .env")
else:
    print("⚠ Warning: .env file not found")
    print("  Create .env from .env.example for secure credential management")

✓ Environment variables loaded from .env


In [4]:
from dataclasses import dataclass
from pathlib import Path
from typing import List


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    """
    Configuration for base model preparation.
    
    Modern practices:
    - Immutable configuration (frozen=True)
    - Type hints for validation
    - Proper Path objects
    """
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: List[int]
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int
    
    def __post_init__(self):
        """Validate configuration"""
        if len(self.params_image_size) != 3:
            raise ValueError("Image size must be [height, width, channels]")
        if self.params_learning_rate <= 0:
            raise ValueError("Learning rate must be positive")
        if self.params_classes < 2:
            raise ValueError("Classes must be >= 2 for classification")

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [6]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories


class ConfigurationManager:
    """Modern configuration manager with validation"""
    
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH
    ):
        """Initialize and validate configuration"""
        try:
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            
            create_directories([self.config.artifacts_root])
            logging.info("✓ Configuration loaded successfully")
            
        except Exception as e:
            logging.error(f"Failed to load configuration: {e}")
            raise

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        """
        Get validated base model configuration.
        
        Returns:
            PrepareBaseModelConfig: Configuration for model preparation
        """
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])
        
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )
        
        logging.info("✓ Base model config created")
        return prepare_base_model_config

In [7]:
import tensorflow as tf
from pathlib import Path
import logging

# Check TensorFlow version
print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {tf.keras.__version__}")

# Configure TensorFlow for optimal performance
# Enable mixed precision for faster training (if GPU available)
if tf.config.list_physical_devices('GPU'):
    print("✓ GPU available - Enabling mixed precision training")
    tf.keras.mixed_precision.set_global_policy('mixed_float16')
else:
    print("⚠ No GPU detected - Using CPU (training will be slower)")

TensorFlow version: 2.20.0
Keras version: 3.12.0
⚠ No GPU detected - Using CPU (training will be slower)


In [8]:
class PrepareBaseModel:
    """
    Modern model preparation class implementing TensorFlow/Keras best practices.
    
    Key improvements:
    - Uses .keras format (Keras 3.0+ standard) instead of deprecated .h5
    - Modern EfficientNet architecture
    - Proper layer freezing strategies
    - Modern optimizer configuration
    - Model validation and summary
    """
    
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        self.logger = logging.getLogger(self.__class__.__name__)
        self.model = None
        self.full_model = None

    def get_base_model(self) -> tf.keras.Model:
        """
        Load EfficientNetB0 pre-trained model.
        
        EfficientNetB0 is chosen because:
        - Modern architecture (2019+, still maintained)
        - Excellent accuracy/efficiency trade-off
        - Better than VGG16 for most tasks
        - 5.3M parameters (lightweight)
        - Works well with small datasets
        
        Returns:
            tf.keras.Model: Pre-trained base model
        """
        try:
            self.logger.info("Loading EfficientNetB0 base model...")
            
            # Load pre-trained EfficientNetB0
            self.model = tf.keras.applications.EfficientNetB0(
                input_shape=self.config.params_image_size,
                weights=self.config.params_weights,
                include_top=self.config.params_include_top
            )
            
            self.logger.info(f"✓ Base model loaded: {self.model.name}")
            self.logger.info(f"  Total parameters: {self.model.count_params():,}")
            
            # Save base model in .keras format (modern Keras 3.0+ standard)
            self.save_model(path=self.config.base_model_path, model=self.model)
            
            return self.model
            
        except Exception as e:
            self.logger.error(f"Failed to load base model: {e}")
            raise

    @staticmethod
    def _prepare_full_model(
        model: tf.keras.Model,
        classes: int,
        freeze_all: bool,
        freeze_till: Optional[int],
        learning_rate: float
    ) -> tf.keras.Model:
        """
        Prepare full model with custom classification head.
        
        Modern best practices:
        - GlobalAveragePooling2D instead of Flatten (fewer parameters)
        - Dropout for regularization (prevent overfitting)
        - BatchNormalization for stable training
        - Modern Adam optimizer
        
        Args:
            model: Base model (e.g., EfficientNetB0)
            classes: Number of output classes
            freeze_all: Whether to freeze all base layers
            freeze_till: Number of layers to freeze from the end
            learning_rate: Learning rate for optimizer
            
        Returns:
            tf.keras.Model: Complete model ready for training
        """
        
        # Configure layer freezing strategy
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
            logging.info("✓ All base model layers frozen")
        elif freeze_till is not None and freeze_till > 0:
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False
            logging.info(f"✓ Frozen {len(model.layers) - freeze_till} layers, training last {freeze_till}")
        else:
            logging.info("✓ All layers trainable (fine-tuning mode)")
        
        # Build modern classification head
        # GlobalAveragePooling2D is better than Flatten (reduces parameters)
        x = tf.keras.layers.GlobalAveragePooling2D(name='global_avg_pool')(model.output)
        
        # Add batch normalization for stable training
        x = tf.keras.layers.BatchNormalization(name='bn_head')(x)
        
        # Dropout for regularization (prevent overfitting on small datasets)
        x = tf.keras.layers.Dropout(0.2, name='dropout_head')(x)
        
        # Optional: Add dense layer before output (uncomment if needed)
        # x = tf.keras.layers.Dense(256, activation='relu', name='dense_head')(x)
        # x = tf.keras.layers.Dropout(0.2, name='dropout_2')(x)
        
        # Output layer
        predictions = tf.keras.layers.Dense(
            units=classes,
            activation='softmax',
            name='output',
            dtype='float32'  # Ensure float32 output for mixed precision
        )(x)
        
        # Create full model
        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=predictions,
            name='EfficientNetB0_ChestCancer'
        )
        
        # Compile with modern optimizer settings
        # Adam optimizer is better than SGD for EfficientNet
        optimizer = tf.keras.optimizers.Adam(
            learning_rate=learning_rate,
            beta_1=0.9,
            beta_2=0.999,
            epsilon=1e-07,
            amsgrad=False,
            name='adam_optimizer'
        )
        
        # Use label smoothing for better generalization
        loss = tf.keras.losses.CategoricalCrossentropy(
            label_smoothing=0.1  # Slight smoothing helps prevent overfitting
        )
        
        full_model.compile(
            optimizer=optimizer,
            loss=loss,
            metrics=[
                'accuracy',
                tf.keras.metrics.AUC(name='auc'),  # Better for imbalanced datasets
                tf.keras.metrics.Precision(name='precision'),
                tf.keras.metrics.Recall(name='recall')
            ]
        )
        
        # Print model summary
        print("\n" + "="*60)
        print("MODEL ARCHITECTURE SUMMARY")
        print("="*60)
        full_model.summary()
        print("="*60 + "\n")
        
        # Print trainable vs non-trainable parameters
        trainable_params = sum([tf.size(w).numpy() for w in full_model.trainable_weights])
        non_trainable_params = sum([tf.size(w).numpy() for w in full_model.non_trainable_weights])
        total_params = trainable_params + non_trainable_params
        
        print(f"Total parameters: {total_params:,}")
        print(f"Trainable parameters: {trainable_params:,}")
        print(f"Non-trainable parameters: {non_trainable_params:,}")
        
        return full_model

    def update_base_model(self) -> tf.keras.Model:
        """
        Update base model with classification head.
        
        Returns:
            tf.keras.Model: Updated model ready for training
        """
        try:
            self.logger.info("Building full model with classification head...")
            
            self.full_model = self._prepare_full_model(
                model=self.model,
                classes=self.config.params_classes,
                freeze_all=True,  # Freeze base for transfer learning
                freeze_till=None,
                learning_rate=self.config.params_learning_rate
            )
            
            self.logger.info("✓ Full model built successfully")
            
            # Save in .keras format (Keras 3.0+ standard, NOT .h5)
            self.save_model(
                path=self.config.updated_base_model_path,
                model=self.full_model
            )
            
            return self.full_model
            
        except Exception as e:
            self.logger.error(f"Failed to update base model: {e}")
            raise

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model) -> None:
        """
        Save model in modern .keras format.
        
        Why .keras format?
        - Keras 3.0+ standard format
        - Single file (not directory)
        - Better compatibility
        - Replaces deprecated .h5 format
        - Supports all modern features
        
        Args:
            path: Path to save model
            model: Model to save
        """
        try:
            # Ensure path has .keras extension
            if not str(path).endswith('.keras'):
                path = Path(str(path).replace('.h5', '.keras'))
                logging.warning(f"Changed extension to .keras: {path}")
            
            # Create parent directory if not exists
            path.parent.mkdir(parents=True, exist_ok=True)
            
            # Save model in .keras format
            model.save(path, save_format='keras')
            
            # Validate saved model
            file_size = path.stat().st_size / (1024 * 1024)  # Convert to MB
            logging.info(f"✓ Model saved: {path} ({file_size:.2f} MB)")
            
        except Exception as e:
            logging.error(f"Failed to save model: {e}")
            raise

In [9]:
# MAIN EXECUTION PIPELINE
# Modern error handling and progress tracking

if __name__ == "__main__":
    try:
        print("\n" + "="*60)
        print("STARTING BASE MODEL PREPARATION")
        print("="*60 + "\n")
        
        # Initialize configuration
        config_manager = ConfigurationManager()
        prepare_base_model_config = config_manager.get_prepare_base_model_config()
        
        # Initialize model preparation
        prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
        
        # Step 1: Load pre-trained base model
        print("Step 1/2: Loading pre-trained EfficientNetB0...")
        base_model = prepare_base_model.get_base_model()
        
        # Step 2: Add classification head and compile
        print("\nStep 2/2: Adding classification head and compiling...")
        full_model = prepare_base_model.update_base_model()
        
        print("\n" + "="*60)
        print("✓ BASE MODEL PREPARATION COMPLETED")
        print("="*60 + "\n")
        print(f"📁 Base model saved: {prepare_base_model_config.base_model_path}")
        print(f"📁 Full model saved: {prepare_base_model_config.updated_base_model_path}")
        print("\n✨ Ready for training!")
        
    except ValueError as e:
        print(f"\n❌ CONFIGURATION ERROR: {e}")
        print("   Check your params.yaml file")
    except Exception as e:
        print(f"\n❌ UNEXPECTED ERROR: {e}")
        import traceback
        traceback.print_exc()
        raise

2025-12-13 00:09:31,646 - cnnClassifierLogger - INFO - yaml file: config\config.yaml loaded successfully
2025-12-13 00:09:31,661 - cnnClassifierLogger - INFO - yaml file: params.yaml loaded successfully
2025-12-13 00:09:31,668 - cnnClassifierLogger - INFO - created directory at: artifacts
2025-12-13 00:09:31,675 - root - INFO - ✓ Configuration loaded successfully
2025-12-13 00:09:31,682 - cnnClassifierLogger - INFO - created directory at: artifacts/prepare_base_model
2025-12-13 00:09:31,687 - root - INFO - ✓ Base model config created
2025-12-13 00:09:31,690 - PrepareBaseModel - INFO - Loading EfficientNetB0 base model...



STARTING BASE MODEL PREPARATION

Step 1/2: Loading pre-trained EfficientNetB0...


2025-12-13 00:09:34,016 - PrepareBaseModel - INFO - ✓ Base model loaded: efficientnetb0
2025-12-13 00:09:34,019 - PrepareBaseModel - INFO -   Total parameters: 4,049,571
2025-12-13 00:09:34,020 - root - WARNING - Changed extension to .keras: artifacts\prepare_base_model\base_model.keras
2025-12-13 00:09:34,021 - absl - WARNING - The `save_format` argument is deprecated in Keras 3. We recommend removing this argument as it can be inferred from the file path. Received: save_format=keras
2025-12-13 00:09:34,977 - root - INFO - ✓ Model saved: artifacts\prepare_base_model\base_model.keras (16.24 MB)
2025-12-13 00:09:34,978 - PrepareBaseModel - INFO - Building full model with classification head...
2025-12-13 00:09:34,983 - root - INFO - ✓ All base model layers frozen



Step 2/2: Adding classification head and compiling...

MODEL ARCHITECTURE SUMMARY


Model: "EfficientNetB0_ChestCancer"

...

 Non-trainable params: 4,052,131 (15.46 MB)

2025-12-13 00:09:35,460 - PrepareBaseModel - INFO - ✓ Full model built successfully
2025-12-13 00:09:35,462 - root - WARNING - Changed extension to .keras: artifacts\prepare_base_model\base_model_updated.keras
2025-12-13 00:09:35,464 - absl - WARNING - The `save_format` argument is deprecated in Keras 3. We recommend removing this argument as it can be inferred from the file path. Received: save_format=keras



Total parameters: 4,057,253
Trainable parameters: 5,122
Non-trainable parameters: 4,052,131


2025-12-13 00:09:36,261 - root - INFO - ✓ Model saved: artifacts\prepare_base_model\base_model_updated.keras (16.28 MB)



✓ BASE MODEL PREPARATION COMPLETED

📁 Base model saved: artifacts\prepare_base_model\base_model.h5
📁 Full model saved: artifacts\prepare_base_model\base_model_updated.h5

✨ Ready for training!
